In [11]:
import jax
from jax import numpy as jnp
import jaxlib
import tensorflow as tf
import numpy as np
import mediapy
from tqdm import tqdm
import dataclasses


from waymax import config as _config
from waymax import dataloader
from waymax.config import DatasetConfig, DataFormat
from waymax import datatypes
from waymax import dynamics
from waymax import env as _env
from waymax import agents
from waymax import visualization
from waymax.dataloader.dataloader_utils import *

In [12]:
path = "./v1_1_0_uncompressed_tf_example_validation_validation_tfexample.tfrecord@150"

files_to_load = generate_sharded_filenames(path)
# print(type(files_to_load))
# for i in files_to_load:
#     print(i)
files = tf.data.Dataset.from_tensor_slices(files_to_load)
# print(files)
# print(type(files))
# for file in files:
#     print(type(file))
# files = tf.data.Dataset.from_tensor_slices(files_to_load)

In [13]:
myconfig = DatasetConfig(
    # path='./training.tfrecord-00100-of-01000',
    # path='gs://waymo_open_dataset_motion_v_1_1_0/uncompressed/tf_example/training/training_tfexample.tfrecord@1000',
    # path="/home/yqnj/testing/testing.tfrecord@150",
    # path = './training.tfrecord@1',
    # path = "./validation.tfrecord@150",
    # path ="./v1_1_0_uncompressed_tf_example_training_training_tfexample.tfrecord-00000-of-01000",
    path = "./v1_1_0_uncompressed_tf_example_validation_validation_tfexample.tfrecord@150",
    max_num_rg_points=20000,
    data_format=DataFormat.TFRECORD,
)
max_num_objects = 8
config = dataclasses.replace(myconfig, max_num_objects=max_num_objects)

In [14]:
data = tf.constant([1, 2, 3, 4, 5])
print(data)
data = tf.data.Dataset.from_tensor_slices(data)
print(data)
for i in data:
    print(i)
data = data.shard(2, 1)

print("=" * 10)

print(data)
for i in data:
    print(i)
print(jax.process_count())
print(jax.process_index())

tf.Tensor([1 2 3 4 5], shape=(5,), dtype=int32)
<_TensorSliceDataset element_spec=TensorSpec(shape=(), dtype=tf.int32, name=None)>
tf.Tensor(1, shape=(), dtype=int32)
tf.Tensor(2, shape=(), dtype=int32)
tf.Tensor(3, shape=(), dtype=int32)
tf.Tensor(4, shape=(), dtype=int32)
tf.Tensor(5, shape=(), dtype=int32)
<_ShardDataset element_spec=TensorSpec(shape=(), dtype=tf.int32, name=None)>
tf.Tensor(2, shape=(), dtype=int32)
tf.Tensor(4, shape=(), dtype=int32)
1
0


In [47]:
filenames = ["/var/data/file1.txt", "/var/data/file2.txt",
             "/var/data/file3.txt", "/var/data/file4.txt"]
dataset = tf.data.Dataset.from_tensor_slices(filenames)
def parse_fn(filename):
  return tf.data.Dataset.range(10)
dataset = dataset.interleave(lambda x:
    tf.data.TextLineDataset(x).map(parse_fn, num_parallel_calls=1),
    cycle_length=4, block_length=16)

Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089


In [64]:
import tensorflow as tf

# 定义一些文件名
filenames = ["file1.txt", "file2.txt", "file3.txt"]

# 创建一个数据集，包含这些文件名
dataset = tf.data.Dataset.from_tensor_slices(filenames)

# 定义一个解析函数，用于将文本行解析成张量
def parse_line(line):
    return tf.strings.split(line)  # 这里仅以空格分割文本行

# 使用interleave函数来交替处理文件中的文本行
# lambda x: tf.data.TextLineDataset(x) 会将文件名转化为对应文件的数据集
# map(parse_line) 将解析函数应用于每个数据集
interleaved_dataset = dataset.interleave(
    lambda x: tf.data.TextLineDataset(x).map(parse_line),
    cycle_length=len(filenames),  # 交替处理的文件数
    num_parallel_calls=tf.data.experimental.AUTOTUNE
)

# 打印前几个元素
for element in interleaved_dataset.take(5):
    print(element)

dataset = tf.data.TextLineDataset("file1.txt")
for i in dataset:
    print(i)

dataset = dataset.repeat(1)

print('=' * 10)
for i in dataset:
    print(i)

tf.Tensor([b'1' b'2' b'3' b'4' b'5'], shape=(5,), dtype=string)
tf.Tensor([b'2' b'4' b'6' b'8' b'10'], shape=(5,), dtype=string)
tf.Tensor([b'4' b'8' b'16' b'32' b'64'], shape=(5,), dtype=string)
tf.Tensor([b'1' b'2' b'3' b'4' b'5'], shape=(5,), dtype=string)
tf.Tensor([b'2' b'4' b'6' b'8' b'10'], shape=(5,), dtype=string)
tf.Tensor(b'1 2 3 4 5', shape=(), dtype=string)
tf.Tensor(b'1 2 3 4 5', shape=(), dtype=string)
tf.Tensor(b'1 2 3 4 5', shape=(), dtype=string)
tf.Tensor(b'1 2 3 4 5', shape=(), dtype=string)
tf.Tensor(b'1 2 3 4 5', shape=(), dtype=string)
tf.Tensor(b'1 2 3 4 5', shape=(), dtype=string)


In [66]:
dataset = tf.data.Dataset.range(1, 6)  # ==> [ 1, 2, 3, 4, 5 ]
# NOTE: New lines indicate "block" boundaries.
dataset = dataset.interleave(
    lambda x: tf.data.Dataset.from_tensors(x).repeat(6),
    cycle_length=2)
list(dataset.as_numpy_iterator())

[1,
 2,
 1,
 2,
 1,
 2,
 1,
 2,
 1,
 2,
 1,
 2,
 3,
 4,
 3,
 4,
 3,
 4,
 3,
 4,
 3,
 4,
 3,
 4,
 5,
 5,
 5,
 5,
 5,
 5]

In [71]:
from waymax.dataloader import womd_utils
myconfig = DatasetConfig(
    path = "./v1_1_0_uncompressed_tf_example_training_training_tfexample.tfrecord-00000-of-01000",
    max_num_rg_points=20000,
    data_format=DataFormat.TFRECORD,
)
data = tf.data.TFRecordDataset("./v1_1_0_uncompressed_tf_example_training_training_tfexample.tfrecord-00000-of-01000")
def preprocess_serialized_womd_data(
    serialized: bytes, config: _config.DatasetConfig
) -> dict[str, tf.Tensor]:
  """Parses serialized tf example into tf Tensor dict."""
  womd_features = womd_utils.get_features_description(
      include_sdc_paths=config.include_sdc_paths,
      max_num_rg_points=config.max_num_rg_points,
      num_paths=config.num_paths,
      num_points_per_path=config.num_points_per_path,
  )

  deserialized = tf.io.parse_example(serialized, womd_features)
  return preprocess_womd_example(
      deserialized,
      aggregate_timesteps=config.aggregate_timesteps,
      max_num_objects=config.max_num_objects,
  )


def preprocess_womd_example(
    example: dict[str, tf.Tensor],
    aggregate_timesteps: bool,
    max_num_objects: Optional[int] = None,
) -> dict[str, tf.Tensor]:
  """Preprocesses dict of tf tensors, keyed by str."""

  if aggregate_timesteps:
    processed = womd_utils.aggregate_time_tensors(example)
    wrap_yaws = lambda yaws: (yaws + jnp.pi) % (2 * jnp.pi) - jnp.pi
    processed['state/all/bbox_yaw'] = wrap_yaws(processed['state/all/bbox_yaw'])
  else:
    processed = example

  if max_num_objects is not None:
    # TODO check sdc included if it is needed.
    return {
        k: v[:max_num_objects] if k.startswith('state/') else v
        for k, v in processed.items()
    }
  else:
    return processed


In [73]:
# preprocess_serialized_womd_data(data, config=myconfig)

preprocess_fn = functools.partial(preprocess_serialized_womd_data, config=myconfig)

data = data.map(
        preprocess_fn, num_parallel_calls=AUTOTUNE, deterministic=True
    )

In [82]:
# print(data)
for i in data:
    # print(i)
    print(len(i.keys()))
    break
# print(len(data.key()))

# cnt = 0
# for i in data:
#     print(cnt)
#     cnt += 1

31
